In [1]:
import pandas as pd
import numpy as np

In [2]:
Df = pd.read_csv('/Users/klm/Downloads/Zing_Customer_Dataset3 - Zing_Customer_Dataset3 (1).csv')

In [3]:
Df.head()

,Customer ID,Gender,Age,Years with Firm,Active on Social Media,Average Transactions per Month,RFM Score,Credit Balance (AED),Customer Bio,Customer Name,Customer Username,Profile Picture on Social Media,Number of Referrals,Payback Status
0,CUST0001,Female,43,9,Yes,13.11,2.0,468,Always learning,Customer 1,user_1,Yes,7,7
1,CUST0002,Male,48,10,No,6.09,1.0,336,Traveler,Customer 2,user_2,No,2,Default
2,CUST0003,Female,19,7,Yes,17.05,1.0,596,Love,Customer 3,user_3,No,3,Default
3,CUST0004,Female,31,0,No,16.77,5.0,488,"Foodie, traveler, and passionate about persona...",Customer 4,user_4,Yes,4,2
4,CUST0005,Male,55,2,No,18.48,2.0,169,Love,Customer 5,customer5,No,0,Default


In [4]:
Df['rfm_category'] = Df['RFM Score'].apply(lambda x: 'Good' if x > 3 else 'Bad')

# Verify the new column
print(Df[['Customer ID', 'RFM Score', 'rfm_category']].head())

  Customer ID  RFM Score rfm_category
0    CUST0001        2.0          Bad
1    CUST0002        1.0          Bad
2    CUST0003        1.0          Bad
3    CUST0004        5.0         Good
4    CUST0005        2.0          Bad


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Convert 'Good'/'Bad' to binary values
Df['rfm_category'] = Df['rfm_category'].map({'Good': 1, 'Bad': 0})

# Select independent variables for the baseline model
X = Df[['Age', 'Years with Firm', 'Average Transactions per Month']]
y = Df['rfm_category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and fit the logistic regression model
model_baseline = LogisticRegression()
model_baseline.fit(X_train, y_train)

# Make predictions
y_pred_baseline = model_baseline.predict(X_test)

# Evaluate the model
accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
print("Baseline Model Accuracy:", accuracy_baseline)

# Confusion Matrix
conf_matrix_baseline = confusion_matrix(y_test, y_pred_baseline)
print("Confusion Matrix for Baseline Model:\n", conf_matrix_baseline)

# Model coefficients
print("Baseline Model Coefficients:\n", model_baseline.coef_)


Baseline Model Accuracy: 0.75
Confusion Matrix for Baseline Model:
 [[135   0]
 [ 45   0]]
Baseline Model Coefficients:
 [[-0.02203664  0.01422655 -0.02709788]]


Improved Model

In [7]:
Df.isnull().sum()  # Check for missing values
# Fill missing values (for example, replacing missing 'Age' with the median)
Df['Age'].fillna(Df['Age'].median(), inplace=True)


In [9]:
# Example for detecting outliers in 'Age'
from scipy import stats
Df = Df[(np.abs(stats.zscore(Df['Age'])) < 3)]  # Remove rows with outliers


In [10]:
Df['Age_Transaction_Interaction'] = Df['Age'] * Df['Average Transactions per Month']


In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Df[['Age', 'Average Transactions per Month']] = scaler.fit_transform(Df[['Age', 'Average Transactions per Month']])


In [13]:
# Adjusting the threshold to 0.6
y_pred_prob = model_baseline.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_pred_prob > 0.6).astype(int)
accuracy_adjusted = accuracy_score(y_test, y_pred_adjusted)
print("Improved Model Accuracy with Adjusted Cutoff:", accuracy_adjusted)


Improved Model Accuracy with Adjusted Cutoff: 0.75


In [14]:
# Select the improved set of independent variables (including interaction terms if needed)
X_improved = Df[['Age', 'Years with Firm', 'Average Transactions per Month', 'Age_Transaction_Interaction']]
y_improved = Df['rfm_category']

# Split data into train and test sets
X_train_improved, X_test_improved, y_train_improved, y_test_improved = train_test_split(X_improved, y_improved, test_size=0.3, random_state=42)

# Train the logistic regression model
model_improved = LogisticRegression()
model_improved.fit(X_train_improved, y_train_improved)

# Make predictions and evaluate the model
y_pred_improved = model_improved.predict(X_test_improved)
accuracy_improved = accuracy_score(y_test_improved, y_pred_improved)
print("Improved Model Accuracy:", accuracy_improved)

# Confusion Matrix
conf_matrix_improved = confusion_matrix(y_test_improved, y_pred_improved)
print("Improved Model Confusion Matrix:\n", conf_matrix_improved)

# Model Coefficients
print("Improved Model Coefficients:\n", model_improved.coef_)


Improved Model Accuracy: 0.75
Improved Model Confusion Matrix:
 [[135   0]
 [ 45   0]]
Improved Model Coefficients:
 [[-0.21267672  0.01541954  0.08672915 -0.00064589]]
